In [1]:
!pip install -U transformers datasets peft accelerate


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
from transformers import TrainingArguments, Trainer

# Choisis un modèle "Causal LM" (par exemple Bloom-560m)
MODEL_NAME = "bigscience/bloom-560m"

# Nom du dataset, ex. "Abirate/english_quotes" (ou autre si tu veux tester)
DATASET_NAME = "Abirate/english_quotes"


ModuleNotFoundError: No module named 'torch'

In [3]:
pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 91.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 26.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip show torch


Name: torch
Version: 2.6.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages
Requires: filelock, fsspec, jinja2, networkx, setuptools, sympy, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install -U peft datasets transformers accelerate


  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 97.3 MB/s eta 0:00:00ta 0:00:01
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 81.7 MB/s eta 0:00:00
Using cached idna-3.10-py3-none-any.whl (70 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
from datasets import load_dataset

# On charge 10% du training set
dataset = load_dataset("Abirate/english_quotes", split="train[:10%]")
print("Taille du dataset :", len(dataset))
print("Exemple :", dataset[0])


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 2508/2508 [00:00<00:00, 260675.88 examples/s]

Taille du dataset : 251
Exemple : {'quote': '“Be yourself; everyone else is already taken.”', 'author': 'Oscar Wilde', 'tags': ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']}


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Charger le modèle pré-entraîné
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)


In [10]:
def tokenize_function(examples):
    return tokenizer(
        examples["quote"], 
        truncation=True,
        max_length=128,
        padding="max_length"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 251/251 [00:00<00:00, 4539.78 examples/s]


In [11]:
tokenized_dataset = tokenized_dataset.remove_columns(["quote", "author"])
tokenized_dataset.set_format("torch")


In [12]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                       # low-rank dimension
    lora_alpha=32,            # alpha (scaling factor)
    target_modules=["query_key_value"],  # couches Bloom à modifier
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(foundation_model, lora_config)
peft_model.print_trainable_parameters()


trainable params: 786,432 || all params: 560,001,024 || trainable%: 0.1404


In [15]:
from transformers import TrainingArguments, Trainer
import torch

def data_collator(batch):
    return {
        "input_ids": torch.stack([f["input_ids"] for f in batch]),
        "attention_mask": torch.stack([f["attention_mask"] for f in batch]),
        "labels": torch.stack([f["input_ids"] for f in batch]),
    }

training_args = TrainingArguments(
    output_dir="peft_lora_outputs",
    num_train_epochs=1,                  # Ajuste si tu veux plus d'époques
    per_device_train_batch_size=4,       # Ajuste selon ta RAM GPU
    gradient_accumulation_steps=1,
    logging_steps=10,
    save_steps=50,
    overwrite_output_dir=True,
    fp16=True,            # si tu as un GPU compatible
    evaluation_strategy="no", 
    # si tu veux évaluer, mets "epoch" ou "steps" et fournis un eval_dataset
    report_to="none"      # ou "tensorboard"
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=None,   # pas de dataset de validation dans l'exemple
    data_collator=data_collator
)


ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [16]:
def tokenize_function(examples):
    return tokenizer(
        examples["quote"],
        truncation=True,
        max_length=128,
        padding="max_length"   # <-- PADDING ICI
    )


In [17]:
from transformers import TrainingArguments, Trainer
import torch

def data_collator(batch):
    return {
        "input_ids": torch.stack([f["input_ids"] for f in batch]),
        "attention_mask": torch.stack([f["attention_mask"] for f in batch]),
        "labels": torch.stack([f["input_ids"] for f in batch]),
    }

training_args = TrainingArguments(
    output_dir="peft_lora_outputs",
    num_train_epochs=1,                  # Ajuste si tu veux plus d'époques
    per_device_train_batch_size=4,       # Ajuste selon ta RAM GPU
    gradient_accumulation_steps=1,
    logging_steps=10,
    save_steps=50,
    overwrite_output_dir=True,
    fp16=True,            # si tu as un GPU compatible
    evaluation_strategy="no", 
    # si tu veux évaluer, mets "epoch" ou "steps" et fournis un eval_dataset
    report_to="none"      # ou "tensorboard"
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=None,   # pas de dataset de validation dans l'exemple
    data_collator=data_collator
)


ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [18]:
def tokenize_function(examples):
    return tokenizer(
        examples["quote"],
        truncation=True,
        max_length=128,
        padding="max_length"  # <-- Le plus important
    )


In [19]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 251/251 [00:00<00:00, 4612.76 examples/s]


In [20]:
def data_collator(batch):
    return {
        "input_ids": torch.stack([f["input_ids"] for f in batch]),
        "attention_mask": torch.stack([f["attention_mask"] for f in batch]),
        "labels": torch.stack([f["input_ids"] for f in batch]),
    }


In [21]:
def tokenize_function(examples):
    return tokenizer(
        examples["quote"],
        truncation=True,
        max_length=128,
        padding="max_length"  # On force le padding ici
    )


In [22]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["quote", "author"])
tokenized_dataset.set_format("torch")


In [23]:
import torch

def data_collator(batch):
    return {
        "input_ids": torch.stack([f["input_ids"] for f in batch]),
        "attention_mask": torch.stack([f["attention_mask"] for f in batch]),
        "labels": torch.stack([f["input_ids"] for f in batch]),
    }


In [24]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./lora_bloom_checkpoint",
    overwrite_output_dir=True,
    num_train_epochs=1,            # Ajuste selon tes besoins
    per_device_train_batch_size=4, # Ajuste selon ta RAM
    logging_steps=50,
    save_steps=100,
    save_total_limit=1,
    evaluation_strategy="no",      # ou "epoch"/"steps" si tu as un eval_dataset
    fp16=True,                     # si tu as un GPU compatible
    report_to="none"
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    # eval_dataset=... (optionnel si tu as un dataset de validation)
    data_collator=data_collator
)


ValueError: fp16 mixed precision requires a GPU (not 'mps').